In [1]:
!nvidia-smi

Tue Dec 30 08:33:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
%%writefile heap_sort.cu

#include <iostream>           // Для cout
#include <vector>             // Для std::vector
#include <cstdlib>            // Для rand()
#include <chrono>             // Для измерения времени
#include <cuda_runtime.h>     // Для работы с CUDA API

using namespace std;          // Чтобы не писать std::

// GPU: функция для «просеивания» (heapify)
__device__ void heapify(int* arr, int n, int i) { // Просеивание элемента i в куче размером n
    int largest = i;            // Инициализируем наибольший элемент как корень
    int l = 2 * i + 1;          // Левый потомок
    int r = 2 * i + 2;          // Правый потомок

    if (l < n && arr[l] > arr[largest]) largest = l; // Сравнение с левым потомком
    if (r < n && arr[r] > arr[largest]) largest = r; // Сравнение с правым потомком

    if (largest != i) {         // Если наибольший не корень
        int temp = arr[i];      // Меняем местами
        arr[i] = arr[largest];
        arr[largest] = temp;
        heapify(arr, n, largest); // Рекурсивно просеиваем
    }
}

// Kernel
// Построение кучи и извлечение элементов параллельно
__global__ void heapSortKernel(int* arr, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x; // Глобальный идентификатор потока

    // Построение кучи (heapify) параллельно для разных корней
    for (int i = n / 2 - 1 - tid; i >= 0; i -= gridDim.x * blockDim.x) {
        heapify(arr, n, i);
    }
    __syncthreads(); // Синхронизация потоков

    // Извлечение элементов из кучи последовательно
    // Полное параллельное извлечение сложно реализовать, поэтому делаем здесь CPU-подобное на GPU
    if (tid == 0) {
        for (int i = n - 1; i >= 0; i--) {
            int temp = arr[0];
            arr[0] = arr[i];
            arr[i] = temp;
            heapify(arr, i, 0);
        }
    }
}

// Main
int main() {
    vector<int> sizes = {10000, 100000};           // Размеры массивов

    for (int s = 0; s < sizes.size(); ++s) {
        int size = sizes[s];
        vector<int> h_arr(size);                   // Массив на CPU

        for (int i = 0; i < size; i++) h_arr[i] = rand() % 100000; // Заполняем случайными числами

        cout << "Исходный массив (первые 20 элементов): ";
        for (int i = 0; i < 20 && i < size; i++) cout << h_arr[i] << " ";
        cout << endl;

        int* d_arr;
        cudaMalloc(&d_arr, size * sizeof(int));    // Выделяем память на GPU
        cudaMemcpy(d_arr, h_arr.data(), size * sizeof(int), cudaMemcpyHostToDevice); // Копируем массив

        int threadsPerBlock = 256;
        int numBlocks = (size + threadsPerBlock - 1) / threadsPerBlock;

        auto startTime = chrono::high_resolution_clock::now();

        heapSortKernel<<<numBlocks, threadsPerBlock>>>(d_arr, size); // Запуск kernel
        cudaDeviceSynchronize();

        auto endTime = chrono::high_resolution_clock::now();
        cudaMemcpy(h_arr.data(), d_arr, size * sizeof(int), cudaMemcpyDeviceToHost); // Копируем результат

        chrono::duration<double> duration = endTime - startTime;

        cout << "Отсортированный массив (первые 20 элементов): ";
        for (int i = 0; i < 20 && i < size; i++) cout << h_arr[i] << " ";
        cout << endl;

        cout << "Размер массива: " << size << endl;
        cout << "Время GPU сортировки: " << duration.count() << " секунд" << endl;

        cudaFree(d_arr);

    }

    return 0;
}


Overwriting heap_sort.cu


In [6]:
# Компиляция и запуск в Colab
!nvcc heap_sort.cu -o heap_sort
!./heap_sort

Исходный массив (первые 20 элементов): 89383 30886 92777 36915 47793 38335 85386 60492 16649 41421 2362 90027 68690 20059 97763 13926 80540 83426 89172 55736 
Отсортированный массив (первые 20 элементов): 89383 30886 92777 36915 47793 38335 85386 60492 16649 41421 2362 90027 68690 20059 97763 13926 80540 83426 89172 55736 
Размер массива: 10000
Время GPU сортировки: 0.00733036 секунд
Исходный массив (первые 20 элементов): 57537 48410 73756 77667 85312 32062 54136 67229 56846 9902 28956 14752 73853 84999 96547 22245 14905 59807 86594 83387 
Отсортированный массив (первые 20 элементов): 57537 48410 73756 77667 85312 32062 54136 67229 56846 9902 28956 14752 73853 84999 96547 22245 14905 59807 86594 83387 
Размер массива: 100000
Время GPU сортировки: 3.7971e-05 секунд
